In [1]:
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots

from functions_hypo_2 import *

In [2]:
df_main = pd.read_csv('data/df_main.csv').drop(columns=["Unnamed: 0"])

In [5]:
# Load demographic data (Population and Area)
df_pop = pd.read_csv("data/eu_country_population.csv")
df_area = pd.read_csv("data/eu_country_area.csv")

# Rename columns
df_pop = df_pop.rename(columns={'OBS_VALUE': "population"})
df_area = df_area.rename(columns={'OBS_VALUE': "area_[km2]"})

# Drop the row for EU27
df_pop = df_pop[df_pop['geo'] != "EU27_2020"]
df_area = df_area[df_area['geo'] != "EU27_2020"]

In [7]:
# Create dataframe with one row per country and aggregate demographic information
df_countries = df_pop[['geo', 'population']].merge(df_area[['geo', 'area_[km2]']], how='inner', on='geo')
country_codes_dict = get_country_codes_dict() # Get dictionary translating country codes to country names
df_countries = df_countries.rename(columns={'geo': 'country_code'}) # Rename country code column
df_countries['country_name'] = df_countries['country_code'].map(country_codes_dict) # Create column with country names

# Add data about solar radiation classification
eu_countries_solar_radiation_classification_dict = get_solar_class_dict() # Get dictionary with classification info
df_countries['solar_class'] = df_countries['country_name'].map(eu_countries_solar_radiation_classification_dict)
df_countries['solar_class_int'] = df_countries['solar_class'].apply(lambda x: x.split(":")[0])

# Add data from df_main (solar electricity production)
df_pivot = df_main.loc[(df_main["product"] == "Solar") & (df_main["year"] > 2013)].pivot_table(index="country_name", values="value", aggfunc="mean")*365
df_countries = df_countries.merge(df_pivot, how='inner', on='country_name')
df_countries = df_countries.rename(columns={'value': 'solar_power_[GWh]'})

# Add data from df_main (total electricity production))
df_pivot = df_main.loc[(df_main["product"] == "Electricity") & (df_main["year"] > 2013)].pivot_table(index="country_name", values="value", aggfunc="mean")*365
df_countries = df_countries.merge(df_pivot, how='inner', on='country_name')
df_countries = df_countries.rename(columns={'value': 'total_annual_electricity_[GWh]'})

# Calculate solar power intensity (production per area)
df_countries['solar_power_intensity_[GWh_per_km2]'] = df_countries['solar_power_[GWh]'] / df_countries["area_[km2]"]
df_countries['solar_power_intensity_[GWh_per_1000_capita]'] = df_countries['solar_power_[GWh]'] / df_countries["population"] * 1000
df_countries['solar_share_in_total_electricity_[%]'] = df_countries['solar_power_[GWh]'] / df_countries["total_annual_electricity_[GWh]"] * 100

In [12]:
df_countries.head(3)

,country_code,population,area_[km2],country_name,solar_class,solar_class_int,solar_power_[GWh],total_annual_electricity_[GWh],solar_power_intensity_[GWh_per_km2],solar_power_intensity_[GWh_per_1000_capita],solar_share_in_total_electricity_[%]
0,AT,9104772,83878,Austria,2: low,2,51382.875000,9.444436e+05,0.612591,5.643510,5.440545
1,BE,11742796,30667,Belgium,2: low,2,138383.666667,1.004391e+06,4.512462,11.784559,13.777871
2,BG,6447710,110996,Bulgaria,3: medium,3,51356.851852,1.262052e+06,0.462691,7.965131,4.069314


In [9]:
fig = make_subplots(rows=3, cols=1, vertical_spacing=0.06,
                    subplot_titles=('Average Annual Solar Net Electricity Production',
                                    '',
                                    ''))

fig.add_trace(go.Bar(
                x=df_countries['country_code'],
                y=df_countries['solar_power_intensity_[GWh_per_km2]']),
                row=1, col=1
                )
fig.add_trace(go.Bar(
                x=df_countries['country_code'],
                y=df_countries['solar_power_intensity_[GWh_per_1000_capita]']),
                row=2, col=1
                )
fig.add_trace(go.Bar(
                x=df_countries['country_code'],
                y=df_countries['solar_share_in_total_electricity_[%]']),
                row=3, col=1
                )

fig.update_layout(height=800, width=800, showlegend=False)

fig.update_xaxes(title_text="Country Code", row=3, col=1)
fig.update_yaxes(title_text="GWh/km²", row=1, col=1)
fig.update_yaxes(title_text="GWh/1000 People", row=2, col=1)
fig.update_yaxes(title_text="Percent in total Electricity", row=3, col=1)

fig.show()

In [10]:
df_pivot_class = df_countries.pivot_table(index="solar_class", values=["solar_power_intensity_[GWh_per_km2]"], aggfunc="mean")

fig = px.bar(df_pivot_class, x=df_pivot_class.index, y="solar_power_intensity_[GWh_per_km2]",
             title='Average Annual Solar Net Electricity Production')

fig.update_layout(height=500, width=800, showlegend=False,
                  title={'y': 0.85, 'x': 0.5, 'xanchor': 'center', 'yanchor': 'top'})

fig.update_xaxes(title_text="Solar Radiation Category")
fig.update_yaxes(title_text="GWh/km²")

fig.show()

In [14]:
fig = px.bar(df_countries.sort_values("solar_class"), x='country_code', y='solar_power_intensity_[GWh_per_km2]',
             color='solar_class',
             labels={'solar_class': 'Radiation Category', 'country_code': "Country Code", 'solar_power_intensity_[GWh_per_km2]': 'Solar Electricity Production per km²'},
             title='Average Annual Solar Net Electricity Production')

fig.update_yaxes(range=[0, 7], title="GWh/km²")
fig.update_xaxes(title_text="Country")
fig.update_layout(height=400, width=800, showlegend=True, title={'y': 0.8, 'x': 0.45, 'xanchor': 'center', 'yanchor': 'top'})

fig.show()
